<a href="https://colab.research.google.com/github/yashas010/ReviewSense/blob/main/C14_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

# Load the new dataset
file_path = "/content/laptop_with_sentiments.csv"  # Update the path if needed
data = pd.read_csv(file_path)

# Check the data structure
print(data.head())


                              Product Name     Price  Rate  \
0  Lenovo Chromebook MediaTek Kompanio 519  ₹ 11,990   5.0   
1  Lenovo Chromebook MediaTek Kompanio 520  ₹ 11,990   5.0   
2  Lenovo Chromebook MediaTek Kompanio 520  ₹ 11,990   5.0   
3  Lenovo Chromebook MediaTek Kompanio 520  ₹ 11,990   5.0   
4  Lenovo Chromebook MediaTek Kompanio 520  ₹ 11,990   5.0   

                                              Review Sentiment  
0  Just Wow! Got it under deal. If you need a lap...  positive  
1  Great machine for regular use. I got the 8/128...  positive  
2  Awesome Laptop. Lenovo Chromebook is best and ...  positive  
3  Great laptop. I loved it. I recommend you to b...  positive  
4                   Easy to use very budget friendly  positive  


In [1]:
import pandas as pd
from transformers import pipeline

# Load the summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to summarize reviews dynamically
def summarize_reviews(reviews):
    """Summarizes reviews dynamically based on their combined length."""
    if not reviews:
        return "No reviews available."

    combined_reviews = " ".join(reviews)
    max_input_length = 1024  # BART model's max token limit

    # Adjust summary length dynamically based on the input length
    if len(combined_reviews) > max_input_length:
        combined_reviews = combined_reviews[:max_input_length]
        max_summary_length = 100
        min_summary_length = 50
    else:
        max_summary_length = 50
        min_summary_length = 20

    summary = summarizer(
        combined_reviews,
        max_length=max_summary_length,
        min_length=min_summary_length,
        do_sample=False
    )
    return summary[0]["summary_text"]

# Function to process and summarize reviews for each product
def process_reviews(data):
    """
    Groups reviews by product and sentiment, then generates summaries.
    """
    summarized_data = []

    grouped_data = data.groupby("Product Name")
    for product, group in grouped_data:
        # Separate positive and negative reviews
        positive_reviews = group[group["Sentiment"] == "positive"]["Review"].tolist()
        negative_reviews = group[group["Sentiment"] == "negative"]["Review"].tolist()

        # Generate summaries
        positive_summary = summarize_reviews(positive_reviews)
        negative_summary = summarize_reviews(negative_reviews)

        # Append summarized results
        summarized_data.append({
            "Product Name": product,
            "Positive Summary": positive_summary,
            "Negative Summary": negative_summary,
        })

    return pd.DataFrame(summarized_data)

# Main script for dynamic input/output
if __name__ == "__main__":
    # Dynamically load data from any CSV file
    input_file = input("Enter the path to your CSV file: ")
    try:
        # Load dataset
        data = pd.read_csv(input_file)
        required_columns = {"Product Name", "Review", "Sentiment"}
        if not required_columns.issubset(data.columns):
            raise ValueError(f"Dataset must contain the following columns: {required_columns}")

        # Process and summarize reviews
        summarized_data = process_reviews(data)

        # Save the summarized results to a new CSV file
        output_file = input("Enter the output file name (e.g., summarized_output.csv): ")
        summarized_data.to_csv(output_file, index=False)
        print(f"Summarized reviews saved to {output_file}.")
    except Exception as e:
        print(f"Error: {e}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Enter the path to your CSV file: /content/laptop_with_sentiments.csv


Your max_length is set to 50, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 50, but your input_length is only 40. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)
Your max_length is set to 50, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_length is set to 50, but your input_length is only 43. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


Enter the output file name (e.g., summarized_output.csv): summary1.csv
Summarized reviews saved to summary1.csv.
